In [1]:
import json

In [3]:
filename = 'demo1.txt'

In [6]:
header_order = {}
answer_count = 0

In [7]:
with open(filename, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            try:
                record = json.loads(line)
                if "answer" in record:
                    answer_count += 1
                    for key in record["answer"].keys():
                        if key not in header_order:
                            header_order[key] = None
            except Exception as e:
                print(f"Error parsing line: {e}")


In [8]:
print(f"Number of 'answer' entries in the file: {answer_count}")

Number of 'answer' entries in the file: 36


In [9]:
print("Headers found in 'answer' (in file order):")
for h in header_order:
    print(h)

Headers found in 'answer' (in file order):
datenow
timenow
rangeIdx
dopplerIdx
numDetectedObj
x_coord
y_coord
z_coord
rp_y
doppz
interFrameProcessingTime
interFrameProcessingMargin
interChirpProcessingMargin
transmitOutputTime
activeFrameCPULoad
interFrameCPULoad
snrDB
noiseDB
activity


In [7]:
import json

filename = 'demo1.txt'

all_rp_y = []
all_doppz = []

with open(filename, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            try:
                record = json.loads(line)
                if "answer" in record:
                    ans = record["answer"]
                    # doppz could be nested (list of lists). Here, we just take the first sequence for animation, adjust if needed.
                    all_doppz.append(ans["doppz"])
                    all_rp_y.append(ans["rp_y"])
            except Exception as e:
                print(f"Error parsing line: {e}")

print(f"Total frames: {len(all_rp_y)}")


Total frames: 36
